In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

# MediaPipe Holistic Model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Directories
PROCESSED_DIR = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/processed_npy"
KEYPOINTS_DIR = "/home/haggenmueller/asl_detection/machine_learning/datasets/own_dataset/keypoints_npy"
os.makedirs(KEYPOINTS_DIR, exist_ok=True)

# Load MediaPipe Holistic
holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=2)

# Function to extract keypoints from a frame
def extract_keypoints(frame):
    frame_rgb = cv2.cvtColor(frame.astype(np.uint8), cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)
    
    keypoints = []
    
    # Face landmarks (x, y, z, visibility=1) → 468 * 4 = 1872
    if results.face_landmarks:
        keypoints.extend([[p.x, p.y, p.z, 1] for p in results.face_landmarks.landmark])
    else:
        keypoints.extend([[0, 0, 0, 1]] * 468)
    
    # Pose landmarks (x, y, z, visibility) → 33 * 4 = 132
    if results.pose_landmarks:
        keypoints.extend([[p.x, p.y, p.z, p.visibility] for p in results.pose_landmarks.landmark])
    else:
        keypoints.extend([[0, 0, 0, 0]] * 33)
    
    # Hand landmarks (x, y, z, visibility=1) → 42 * 4 = 168
    for hand_landmarks in [results.left_hand_landmarks, results.right_hand_landmarks]:
        if hand_landmarks:
            keypoints.extend([[p.x, p.y, p.z, 1] for p in hand_landmarks.landmark])
        else:
            keypoints.extend([[0, 0, 0, 1]] * 21)
    
    return np.array(keypoints).flatten()

# Process all .npy videos
for npy_file in tqdm(os.listdir(PROCESSED_DIR)):
    if not npy_file.endswith(".npy"):
        continue
    
    video_path = os.path.join(PROCESSED_DIR, npy_file)
    frames = np.load(video_path)
    
    keypoints_sequence = []
    
    for frame in frames:
        frame = frame.transpose(1, 2, 0)  # Convert [C, H, W] -> [H, W, C]
        keypoints = extract_keypoints(frame)
        keypoints_sequence.append(keypoints)
    
    keypoints_sequence = np.array(keypoints_sequence)
    
    # Save keypoints
    keypoints_path = os.path.join(KEYPOINTS_DIR, npy_file)
    np.save(keypoints_path, keypoints_sequence)
    print(f"✅ Saved keypoints for {npy_file}")

print("🚀 Keypoints extraction completed!")

I0000 00:00:1741554760.665387 1368867 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741554760.739206 1369139 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.216.01), renderer: NVIDIA RTX A6000/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1741554760.834157 1369078 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741554760.923600 1369086 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741554760.931467 1369090 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741554760.934263 1369097 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Di

✅ Saved keypoints for 50037_secretary_44.npy


  0%|                                                                                                                              | 2/8360 [00:20<24:58:33, 10.76s/it]

✅ Saved keypoints for 00314_sleep_89.npy


  0%|                                                                                                                              | 3/8360 [00:27<21:05:06,  9.08s/it]

...

✅ Saved keypoints for 00149_have_72_noise.npy


  0%|                                                                                                                              | 8/8360 [01:19<24:57:07, 10.76s/it]

✅ Saved keypoints for 40119_orange_88_slow.npy


  0%|▏                                                                                                                             | 9/8360 [01:31<25:14:48, 10.88s/it]

✅ Saved keypoints for 38524_no_80_brightness.npy
